# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
!jupyter nbextension enable --py gmaps

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
weather_data = pd.read_csv(r'../WeatherPy/WeatherPy.csv')
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kakonko,-3.28,30.96,68.79,52,25,2.86,TZ,1598736704
1,Leningradskiy,69.38,178.42,36.84,88,100,10.13,RU,1598736705
2,Presidencia Roque Sáenz Peña,-26.79,-60.44,83.91,45,97,11.30,AR,1598736705
3,Rikitea,-23.12,-134.97,70.03,68,58,6.98,PF,1598736705
4,Camacha,33.08,-16.33,71.60,64,40,12.75,PT,1598736706


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
gmaps.configure(api_key=g_key)

In [17]:
fig = gmaps.figure()
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
cond_weather_data = weather_data.loc[(weather_data["Max Temp"]>=75) & (weather_data["Max Temp"]<=80) & (weather_data["Humidity"]<50) & (weather_data["Wind Speed"]<10)]
cond_weather_data.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
299,Portland,45.52,-122.68,78.01,40,40,9.17,US,1598736517
300,Beitbridge,-22.22,30.00,79.56,24,82,0.74,ZW,1598736793
311,Pandamatenga,-18.53,25.63,75.43,35,85,0.85,BW,1598736797
381,Manbij,36.53,37.95,78.80,36,0,6.87,SY,1598736816
407,Siirt,37.93,41.94,77.11,20,0,3.89,TR,1598736825
456,Chanthaburi,12.83,102.17,79.00,10,30,1.92,TH,1598736840
556,Scalea,39.81,15.80,79.00,34,0,3.60,IT,1598736869


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = []
cond_weather_data['Hotel Name'] = ""
cond_weather_data.head()

<ipython-input-19-999f92e64ec3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cond_weather_data['Hotel Name'] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
299,Portland,45.52,-122.68,78.01,40,40,9.17,US,1598736517,
300,Beitbridge,-22.22,30.00,79.56,24,82,0.74,ZW,1598736793,
311,Pandamatenga,-18.53,25.63,75.43,35,85,0.85,BW,1598736797,
381,Manbij,36.53,37.95,78.80,36,0,6.87,SY,1598736816,
407,Siirt,37.93,41.94,77.11,20,0,3.89,TR,1598736825,


In [ ]:
for City in cond_weather_data[0:9]:
    
    target_coordinates = str(City["Lat"] + "," + str(City["Lng"])
    target_search = "Hotel"
    target_radius = 5000
    target_type = "hotel"
                             
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
    }
                             
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
                             
request = requests.get(base_url, params=params)
response = request.json()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
#https://jupyter-gmaps.readthedocs.io/en/v0.3.4/gmaps.html
marker_locations = cond_weather_data['Hotel Name']
markers = gmaps.marker_layer(marker_locations)

m = gmaps.Map()
m.add_layer(markers)
m
# Display figure
